In [ ]:
import os
import json
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:

def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

messages = [{"role": "user", "content": "지금 서울날씨를 섭씨로 알려줘."}]
functions = [
    {
        "name": "get_current_weather",
        "description": "특정 지역의 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "지역이름. 예) 서울, 부산, 제주도",
                },
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto",
    )
response_message = response.choices[0].message
print(response_message)

In [ ]:
if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        location=function_args['location'],
        unit=function_args['unit']
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

    print(second_response.choices[0].message.content)

In [ ]:
messages = [{"role": "user", "content": "지금 대전 날씨를 섭씨로 알려줘."}]
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_current_weather",
            "description": "특정 지역의 날씨를 알려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "지역이름. 예) 서울, 부산, 제주도",
                    },
                    "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
                },
                "required": ["location"],
            },
        }
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto" if tools is not None else None
    )
response_message = response.choices[0].message
print(response_message)

In [ ]:
available_functions = {
    "get_current_weather": get_current_weather,
}

for tool_call in response_message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args['location'],
        unit=function_args['unit']
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)
print(second_response.choices[0].message.content)